<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/dynamic_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [7]:
alpha=5.
x=np.linspace(0,1,100)
y=x*alpha/(1-x+x*alpha)
fig=make_subplots()
fig.add_scatter(x=x,y=y,mode='lines')
fig.update_layout(width=600,height=600, template='plotly_dark')

In [24]:
NT=3
NF=2
weir_constant = 2.
F = 1.
zF = 0.6
q = 0.5

D = 0.55
R = 2

B=F-D

Kc = 0.1
tray_m = 0.1  #molar holdup on tray
reboiler_m_sp = 0.5
condenser_m = 0.5

Vrec = (R+1)*D
Vstrip = Vrec- (1-q)*F
V=np.r_[np.repeat(Vstrip,NF), np.repeat(Vrec, NT-NF+1), 0.] #total condenser

L_initial = np.r_[B, np.repeat(Vstrip - B, NF), np.repeat(Vrec - D, NT-NF+1)]

m_initial = np.r_[reboiler_m_sp, tray_m+(L_initial[1:-1]/weir_constant)**(2/3), condenser_m]

tend=10.

In [14]:
def rhs(t,vec):
    m, x = np.split(vec,2)
    y=x*alpha/(1-x+x*alpha)

    dm=np.zeros_like(m)
    dx = np.zeros_like(x)
    dmx=np.zeros_like(m)
    L = np.zeros_like(m)

    L[-1] = R*D
    L[0] =  F-D + Kc*(m[0]- reboiler_m_sp)

    L[1:-1] = weir_constant*(m[1:-1] - tray_m)**1.5
    dm[1:-1] = V[:-2]+L[2:] - V[1:-1] - L[1:-1]
    dmx[1:-1]= y[:-2]*V[:-2] + x[2:]*L[2:] - y[1:-1]*V[1:-1] - x[1:-1]*L[1:-1]

    dm[NF]+=F
    dmx[NF]+=zF*F

    dm[-1] = V[-2] - L[-1] - D
    dmx[-1]= y[-2]*V[-2] - x[-1]*L[-1] - x[-1]*D

    dm[0] = L[1] - V[0] - L[0]
    dmx[0]= x[1]*L[1] - y[0]*V[0] - x[0]*L[0]

    dx = (x*dm - dmx)/m

    return np.r_[dm, dx]


In [25]:
np.r_[np.repeat(zF,NT+2), m_initial]

array([0.6       , 0.6       , 0.6       , 0.6       , 0.6       ,
       0.5       , 0.59664419, 0.59664419, 0.77128734, 0.5       ])

In [21]:
solve_ivp(rhs, (0,tend), np.r_[np.repeat(zF,NT+2), m_initial], method='Radau',dense_output=True)

  message: Required step size is less than spacing between numbers.
  success: False
   status: -1
        t: [ 0.000e+00  4.109e-02 ...  2.181e+00  2.181e+00]
        y: [[ 6.000e-01  5.354e-01 ... -2.473e+00 -2.473e+00]
            [ 6.000e-01  6.000e-01 ...  7.735e-01  7.735e-01]
            ...
            [ 5.046e+00  5.394e+00 ...  8.811e+01  8.811e+01]
            [ 5.000e-01  4.171e-01 ... -2.846e+02 -2.846e+02]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7d96e7ed7790>
 t_events: None
 y_events: None
     nfev: 1061
     njev: 45
      nlu: 246